<a href="https://colab.research.google.com/github/miguelmccormickudg/Gravitational_housing_model/blob/main/notebook_red_neural_ya_probada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook completo
# Predicción de atracción gravitacional entre propiedades y zonas de transformación territorial con redes neuronales + DENUE
# 0. Instalación de librerías y setup

!pip install geopandas shapely simplekml

import pandas as pd
import numpy as np
import geopandas as gpd
import tensorflow as tf
from itertools import combinations
from math import radians, sin, cos, asin, sqrt
import simplekml
import shapely.geometry

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Carga de archivos base

# Rutas locales (como las tienes ahora):

# Archivos principales del modelo
PATH_PROPS   = "/content/resultados_gravedad_propiedades_v2.xlsx"
PATH_GRAV    = "/content/gravedad.csv"
PATH_INFL    = "/content/influence_summary_cat3to5.csv"
PATH_DENUE   = "/content/denue.xlsx"

# Cargar tablas
props = pd.read_excel(PATH_PROPS)
grav  = pd.read_csv(PATH_GRAV)
infl  = pd.read_csv(PATH_INFL)
denue = pd.read_excel(PATH_DENUE)

print("props:", props.shape)
print("grav :", grav.shape)
print("infl :", infl.shape)
print("denue:", denue.shape)


# 2. Estandarización de columnas y fusiones
# 2.1. Propiedades, gravedad y radio de influencia

# Normalizar nombres en props
props = props.rename(columns={
    "coordinate/latitude":  "lat",
    "coordinate/longitude": "lon",
    "Masa": "mass",
    "FuerzaTotal": "F_total"
})

# Normalizar nombres en gravedad
grav = grav.rename(columns={
    "coordinate/latitude":  "lat",
    "coordinate/longitude": "lon",
    "F_i": "F_node"   # nombre estándar para F_i de nodo
})

# Normalizar nombres en influencia
infl = infl.rename(columns={
    "gravity_value": "F_inf"
})

# Redondeo de coordenadas para merge aproximado
def round_coords(df, ndigits=5):
    df["lat_r"] = df["lat"].round(ndigits)
    df["lon_r"] = df["lon"].round(ndigits)
    return df

props = round_coords(props)
grav  = round_coords(grav)
infl  = round_coords(infl)

# Merge con gravedad
props = props.merge(
    grav[["lat_r","lon_r","S_i","entropy_weight","F_node"]],
    on=["lat_r","lon_r"],
    how="left"
)

# Merge con resumen de influencia (radio R_influence_km)
props = props.merge(
    infl[["lat_r","lon_r","R_influence_km"]],
    on=["lat_r","lon_r"],
    how="left"
)

# Rellenar NaN con valores razonables
props["R_influence_km"] = props["R_influence_km"].fillna(props["R_influence_km"].median())
props["S_i"]            = props["S_i"].fillna(props["S_i"].median())
props["F_node"]         = props["F_node"].fillna(props["F_node"].median())

print("props tras merges:", props.shape)


# 2.2. Preparación de DENUE

# Columnas reales de tu DENUE:

print(denue.columns)


# Tienen latitud, longitud, nombre_act. Las normalizamos:

denue = denue.rename(columns={
    "latitud": "lat",
    "longitud": "lon",
    "nombre_act": "actividad"
})

print("DENUE columnas normalizadas:", denue[["lat","lon","actividad"]].head())


# 3. GeoDataFrames y proyección

# Usaremos un CRS métrico local para Chapala. Ajusta el EPSG si necesitas otro, pero este patrón es válido.

# Propiedades
gdf_props = gpd.GeoDataFrame(
    props,
    geometry=gpd.points_from_xy(props["lon"], props["lat"]),
    crs="EPSG:4326"
).to_crs("EPSG:6372")   # CRS métrico local (ajustable)

# DENUE
gdf_denue = gpd.GeoDataFrame(
    denue,
    geometry=gpd.points_from_xy(denue["lon"], denue["lat"]),
    crs="EPSG:4326"
).to_crs(gdf_props.crs)

print("gdf_props:", gdf_props.shape)
print("gdf_denue:", gdf_denue.shape)


# 4. Variables territoriales DENUE por propiedad

# Calculamos para cada propiedad:

# denue_total_500m: número de establecimientos a 500 m

# denue_turismo_500m: establecimientos turísticos (hotel, alojamiento, restaurantes, bares)

# denue_diversidad_500m: número de tipos de actividad distintos

buffer_radius = 500  # metros

def es_turismo(act):
    if isinstance(act, str):
        act_low = act.lower()
        if "hotel" in act_low or "alojamiento" in act_low or "hospedaje" in act_low:
            return 1
        if "restaurante" in act_low or "restaurant" in act_low or "bar" in act_low:
            return 1
    return 0

gdf_denue["turistico"] = gdf_denue["actividad"].apply(es_turismo)

denue_total = []
denue_tur   = [] # Changed from denu_tur to denue_tur
denue_div   = [] # Changed from denu_div to denue_div

for geom in gdf_props.geometry:
    buf = geom.buffer(buffer_radius)
    df_loc = gdf_denue[gdf_denue.geometry.within(buf)]

    denue_total.append(len(df_loc))
    denue_tur.append(df_loc["turistico"].sum())
    denue_div.append(df_loc["actividad"].nunique())

props["denue_total_500m"]      = denue_total
props["denue_turismo_500m"]    = denue_tur
props["denue_diversidad_500m"] = denue_div

print("Variables DENUE añadidas a props.")


# 5. Función Haversine

def haversine(lat1, lon1, lat2, lon2, R=6371):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*asin(sqrt(a))
    return R*c  # km


# 6. Dataset de pares (i, j) y etiqueta Fij

# Definimos:

# # 𝐹
# # 𝑖
# # 𝑗
# # 𝑙
# # 𝑎
# # 𝑏
# # 𝑒
# # 𝑙
# # =
# # 𝐹
# # 𝑖
# # +
# # 𝐹
# # 𝑗
# # 2
# # ⋅
# # 1
# # 1
# # +
# # 𝑑
# # 𝑖
# # 𝑠
# # 𝑡
# # 𝑖
# # 𝑗
# # 𝛽
# # F
# # ij
# # label
# #

# # =
# # 2
# # F
# #

# # +F
# # j
# #

# #

# # ⋅
# # 1+dist
# # ij
# # β
# #

# # 1
# #


# y una etiqueta binaria de atracción según R_influence_km de la propiedad i.

pairs = []
beta = 2.0
max_radius_km = 3.0  # distancia máxima para considerar pares

indices = props.index.to_list()

for i, j in combinations(indices, 2):
    pi = props.loc[i]
    pj = props.loc[j]

    dist_ij = haversine(pi["lat"], pi["lon"], pj["lat"], pj["lon"])
    if dist_ij > max_radius_km:
        continue

    Fi = pi["F_node"] if not np.isnan(pi["F_node"]) else pi["F_total"]
    Fj = pj["F_node"] if not np.isnan(pj["F_node"]) else pj["F_total"]

    Fij_label = ((Fi + Fj) / 2.0) * (1.0 / (1.0 + dist_ij**beta))

    attract_ij = 1 if dist_ij <= pi["R_influence_km"] else 0

    row = {
        "i": i,
        "j": j,
        "dist_ij": dist_ij,
        "mass_i": pi["mass"],
        "mass_j": pj["mass"],
        "F_i": Fi,
        "F_j": Fj,
        "S_i": pi["S_i"],
        "S_j": pj["S_i"],

        # variables DENUE por propiedad
        "denue_total_i":      pi["denue_total_500m"],
        "denue_total_j":      pj["denue_total_500m"],
        "denue_turismo_i":    pi["denue_turismo_500m"],
        "denue_turismo_j":    pj["denue_turismo_500m"],
        "denue_div_i":        pi["denue_diversidad_500m"],
        "denue_div_j":        pj["denue_diversidad_500m"],

        "Fij_label": Fij_label,
        "attract_label": attract_ij
    }
    pairs.append(row)

pairs_df = pd.DataFrame(pairs)
pairs_df = pairs_df.fillna(pairs_df.median(numeric_only=True))

print("Total de pares generados:", len(pairs_df))


# 7. MODELO 1: Red neuronal Fij + Atracción entre propiedades
# 7.1 Preparar features y escalado

feature_cols = [
    "dist_ij",
    "mass_i","mass_j",
    "F_i","F_j",
    "S_i","S_j",
    "denue_total_i","denue_total_j",
    "denue_turismo_i","denue_turismo_j",
    "denue_div_i","denue_div_j"
]

X = pairs_df[feature_cols].values
y_reg = pairs_df["Fij_label"].values
y_cls = pairs_df["attract_label"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Xtr, Xte, yr_tr, yr_te, yc_tr, yc_te = train_test_split(
    X_scaled, y_reg, y_cls, test_size=0.2, random_state=42
)

Xtr.shape, Xte.shape


# 7.2 Definir y entrenar modelo multitarea

inputs = tf.keras.Input(shape=(Xtr.shape[1],))

x = tf.keras.layers.Dense(64, activation="relu")(inputs)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(16, activation="relu")(x)

out_reg = tf.keras.layers.Dense(1, name="Fij")(x)
out_cls = tf.keras.layers.Dense(1, activation="sigmoid", name="Attr")(x)

model1 = tf.keras.Model(inputs=inputs, outputs=[out_reg, out_cls])

model1.compile(
    optimizer="adam",
    loss={"Fij": "mse", "Attr": "binary_crossentropy"},
    metrics={"Fij": "mae", "Attr": "accuracy"}
)

history1 = model1.fit(
    Xtr,
    {"Fij": yr_tr, "Attr": yc_tr},
    validation_data=(Xte, {"Fij": yr_te, "Attr": yc_te}),
    epochs=40,
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)


# 8. KML de pares de propiedades atraídas

from simplekml import Kml, Color

# Verificamos existencia de columnas necesarias y ejecutamos predicción si es necesario
if "attr_prob" not in pairs_df.columns or "Fij_pred" not in pairs_df.columns:
    pred_fij, pred_attr = model1.predict(X_scaled, verbose=0)
    pairs_df["Fij_pred"] = pred_fij.flatten()
    pairs_df["attr_prob"] = pred_attr.flatten()

# Umbral de probabilidad
thr_prob = 0.5
high_pairs = pairs_df[pairs_df["attr_prob"] >= thr_prob].copy()

# Limitar máximo a 5000 líneas por razones de rendimiento
high_pairs = high_pairs.head(5000)

if high_pairs.empty:
    print(f"No hay pares con probabilidad >= {thr_prob}")
else:
    kml1 = Kml()
    errores = 0
    for _, row in high_pairs.iterrows():
        try:
            pi = props.loc[int(row["i"])]
            pj = props.loc[int(row["j"])]

            if any(pd.isna([pi["lat"], pi["lon"], pj["lat"], pj["lon"]])):
                continue

            ls = kml1.newlinestring(
                name=f"p={row['attr_prob']:.2f}",
                coords=[(pi["lon"], pi["lat"]), (pj["lon"], pj["lat"])]
            )
            ls.style.linestyle.width = 2
            ls.style.linestyle.color = Color.blue
        except Exception as e:
            errores += 1
            continue

    kml1.save("atraccion_propiedades_fij_denue.kml")
    print(f"KML generado correctamente con {len(high_pairs)-errores} líneas. Errores: {errores}")


# 9. MODELO 2: Zonas de transformación territorial (vectorial + DENUE)
# 9.1 Construcción de grilla espacial

minx, miny, maxx, maxy = gdf_props.to_crs(gdf_props.crs).total_bounds
cell_size = 150  # metros

xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)

polygons = []
for x in xs:
    for y in ys:
        polygons.append(shapely.geometry.box(x, y, x+cell_size, y+cell_size))

gdf_grid = gpd.GeoDataFrame(geometry=polygons, crs=gdf_props.crs)

print("Celdas en grilla:", gdf_grid.shape)

# Removed redundant feature preparation and check from here


# 9.2 Agregar propiedades a la grilla

join_props = gpd.sjoin(gdf_props, gdf_grid, how="left", predicate="within")

agg_props = join_props.groupby("index_right").agg({
    "mass": "sum",
    "F_total": "sum",
    "S_i": "mean",
    "F_node": "mean"
}).rename(columns={
    "mass": "mass_sum",
    "F_total": "F_sum",
    "S_i": "S_mean",
    "F_node": "Fnode_mean"
})

gdf_grid = gdf_grid.join(agg_props, how="left")
gdf_grid[["mass_sum","F_sum","S_mean","Fnode_mean"]] = gdf_grid[["mass_sum","F_sum","S_mean","Fnode_mean"]].fillna(0)


# 9.3 Agregar DENUE a la grilla

join_denue = gpd.sjoin(gdf_denue, gdf_grid, how="left", predicate="within")

agg_denue = join_denue.groupby("index_right").agg({
    "actividad": "nunique",
    "turistico": "sum"
}).rename(columns={
    "actividad": "denue_div_grid",
    "turistico": "denue_turismo_grid"
})

gdf_grid = gdf_grid.join(agg_denue, how="left")
gdf_grid[["denue_div_grid","denue_turismo_grid"]] = gdf_grid[["denue_div_grid","denue_turismo_grid"]].fillna(0)

print("gdf_grid con props + DENUE:", gdf_grid.shape)


# 9.4 ITT y clases territoriales

a1, a2, a3, a4 = 1.0, 0.5, 0.3, 0.4

gdf_grid["ITT"] = (
    a1*gdf_grid["F_sum"] +
    a2*gdf_grid["mass_sum"] -
    a3*gdf_grid["S_mean"] +
    a4*gdf_grid["denue_turismo_grid"]
)

gdf_grid = gdf_grid.copy()
gdf_grid["ITT_rank"] = gdf_grid["ITT"].rank(pct=True)

def classify_balanced(rank):
    if rank <= 0.33:
        return 0
    elif rank <= 0.66:
        return 1
    else:
        return 2

gdf_grid["ITT_class"] = gdf_grid["ITT_rank"].apply(classify_balanced)
print("Distribución de clases recalculada:")
print(gdf_grid["ITT_class"].value_counts())


gdf_grid[["ITT","ITT_class"]].head()


# 9.5 MLP vectorial para ITT + clase

features_grid = [
    "mass_sum","F_sum","S_mean","Fnode_mean",
    "denue_div_grid","denue_turismo_grid"
]

Xg = gdf_grid[features_grid].values
yI = gdf_grid["ITT"].values
yC = gdf_grid["ITT_class"].values

scaler_g = StandardScaler()
Xg_scaled = scaler_g.fit_transform(Xg)

Xg_tr, Xg_te, yI_tr, yI_te, yC_tr, yC_te = train_test_split(
    Xg_scaled, yI, yC, test_size=0.2, random_state=42
)

inp_g = tf.keras.Input(shape=(Xg_tr.shape[1],))
z = tf.keras.layers.Dense(32, activation="relu")(inp_g)
z = tf.keras.layers.Dense(16, activation="relu")(z)
z = tf.keras.layers.Dropout(0.2)(z)

ITT_out   = tf.keras.layers.Dense(1, name="ITT_out")(z)
Class_out = tf.keras.layers.Dense(3, activation="softmax", name="Class_out")(z)

model2 = tf.keras.Model(inp_g, [ITT_out, Class_out])

model2.compile(
    optimizer="adam",
    loss={"ITT_out": "mse", "Class_out": "sparse_categorical_crossentropy"},
    metrics={"ITT_out": "mae", "Class_out": "accuracy"}
)

history2 = model2.fit(
    Xg_tr,
    {"ITT_out": yI_tr, "Class_out": yC_tr},
    validation_data=(Xg_te, {"ITT_out": yI_te, "Class_out": yC_te}),
    epochs=40,
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)


# 9.6 Predicciones y KML de zonas de transformación

import simplekml

# Verificar y forzar predicción si hace falta
if "ITT_pred" not in gdf_grid.columns or "Class_pred" not in gdf_grid.columns:
    Xg_all = scaler_g.transform(gdf_grid[features_grid].values)
    ITT_pred_all, Class_pred_all = model2.predict(Xg_all, verbose=0)
    gdf_grid["ITT_pred"]   = ITT_pred_all.flatten()
    gdf_grid["Class_pred"] = Class_pred_all.argmax(axis=1)
print("Resumen de clases predichas:")
print(gdf_grid["Class_pred"].value_counts())


# Diagnóstico
print("Valores únicos en predicción:", gdf_grid["Class_pred"].value_counts().to_dict())

# Filtrar clase 2
zonas_altas = gdf_grid[gdf_grid["Class_pred"] == 2].copy()

if zonas_altas.empty:
    print("Advertencia: no se encontraron zonas de transformación territorial clase 2.")
else:
    kml2 = simplekml.Kml()
errores = 0

for _, row in zonas_altas.iterrows():
    try:
        poly = row.geometry
        if poly and hasattr(poly, "exterior"):
            pol = kml2.newpolygon()
            pol.outerboundaryis = list(poly.exterior.coords)
            pol.style.polystyle.color = simplekml.Color.changealphaint(120, simplekml.Color.red)
            pol.name = f"Clase {row['Class_pred']}, ITT={row['ITT_pred']:.2f}"
    except Exception as e:
        errores += 1
        print(f"Error al procesar polígono: {e}")

kml2.save("zonas_transformacion_denue.kml")
print(f"KML generado con {len(zonas_altas) - errores} zonas. Errores: {errores}")



# 10. CNN espacial (opcional) para validación cruzada

# Si quieres la parte raster / CNN, puedes añadir esta sección. Si por ahora quieres mantenerlo simple, puedes omitirla.

# Asignar índices de fila/columna a cada celda de la grilla
gdf_grid["centroid"] = gdf_grid.geometry.centroid
minx_c, miny_c, maxx_c, maxy_c = gdf_grid.total_bounds

gdf_grid["row"] = ((gdf_grid["centroid"].y - miny_c) // cell_size).astype(int)
gdf_grid["col"] = ((gdf_grid["centroid"].x - minx_c) // cell_size).astype(int)

max_row = gdf_grid["row"].max() + 1
max_col = gdf_grid["col"].max() + 1

channels = features_grid  # mismas variables de entrada
grid_tensor = np.zeros((max_row, max_col, len(channels)), dtype=float)
label_tensor = np.zeros((max_row, max_col), dtype=int)

for _, r in gdf_grid.iterrows():
    i = r["row"]
    j = r["col"]
    for c_idx, c_name in enumerate(channels):
        grid_tensor[i, j, c_idx] = r[c_name]
    label_tensor[i, j] = r["ITT_class"]

# Normalización por canal
for c_idx in range(len(channels)):
    ch = grid_tensor[:, :, c_idx]
    mu = ch.mean()
    sigma = ch.std() if ch.std() > 0 else 1.0
    grid_tensor[:, :, c_idx] = (ch - mu) / sigma

# Añadir dimensión batch
X_cnn = np.expand_dims(grid_tensor, axis=0)
y_cnn = np.expand_dims(label_tensor, axis=0)

from tensorflow.keras import layers, models

input_shape = X_cnn.shape[1:]

inputs_cnn = layers.Input(shape=input_shape)
y = layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs_cnn)
# Removed MaxPooling2D layers to maintain spatial resolution
y = layers.Conv2D(64, (3,3), activation="relu", padding="same")(y)
y = layers.Conv2D(64, (3,3), activation="relu", padding="same")(y)
y = layers.Dropout(0.3)(y) # Apply dropout to feature maps
# Replaced Flatten and Dense layers with a Conv2D layer for pixel-wise classification
outputs_cnn = layers.Conv2D(3, kernel_size=(3,3), activation="softmax", padding="same")(y)

model_cnn = models.Model(inputs_cnn, outputs_cnn)
model_cnn.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_cnn.fit(X_cnn, y_cnn, epochs=20, batch_size=1)


props: (998, 41)
grav : (320, 8)
infl : (190, 8)
denue: (2960, 43)
props tras merges: (2680, 47)
Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta', 'anio_alta'],
      dtype='object')
DENUE columnas normalizadas:          lat         lon                                          actividad
0  20.300415 -103.194913  Comercio al por menor de paletas de hielo y he...
1  20.288974 -103.193082                Comercio al por menor en minisupers
2  20.296308 -103.243470        